# Einfügen und Löschen von Elementen

<div class="prereq">
    <h3>Was man wissen sollte</h3>
    <div>
        Wir benötigen
        <a class="prereq" href="/user-redirect/algoviz/lessons/06_ADT/02_Iteration.ipynb">Iteratoren</a>
        für Listen.
    </div>
</div>

<div class="slideshow 06_ADT/03_EinfuegenLoeschen/slides.json">Einfügen und Löschen in Listen</a>

Die Implementierung einer Liste in C++ ermöglicht das Löschen und Einfügen mit den Methoden `erase()` und `insert()`. Erstere erhält nur einen Iterator als Parameter, während die zweite einen Iterator und ein einzufügendes Element erhält. Beide geben außerdem einen Iterator als Ergebnis zurück.

## Einfügen

In [21]:
#include <iostream>
#include <list>
using namespace std;

list<int> liste = list<int>();

liste.push_front(1);
liste.push_front(2);
liste.push_front(3);     // Die Liste hat jetzt den Inhalt (3,2,1)

auto it = liste.begin(); // Gehe zum ersten Element 
it++;
it++;                    // Jetzt ist der Iterator bei der 1

auto it2 = liste.insert(it,4);  // Die 4 wird VOR der 1 eingefügt: (3,2,4,1)

for ( int i : liste ) {
    cout << i << " ";
}

3 2 4 1 

Wie angekündigt wurde die 4 vor der 1 eingefügt. Serhen wir uns noch an, wo der ursprüngliche Iterator nach dem Einfügen steht und auf welches Element der zurückgegebene Iterator `it2` zeigt.

In [22]:
cout << *it;

1

Der ursprüngliche Iterator wurde nicht verändert. Er zeigt auf das alte Element.

In [23]:
cout << *it2;

4

Der **zurückgegebene** Iterator zeigt auf das neue Element.

Fassen wir zusammen. `liste.insert(it,v)` fügt den Wert `v` **vor** dem Element auf das der Iterator `it` zeigt, ein. Der übergebene Iterator bleibt unverändert. Aber `insert()` gibt einen Iterator zurück, der auf das **neue** Element zeigt.

## Löschen

Kommen wir jetzt zum Löschen.

In [1]:
it = liste.begin();   // Zur 3
it++;                 // Zur 2

it2 = liste.erase(it);      // Die 2 wird gelöscht.
    
for ( int i : liste ) {
    cout << i << " ";
}

input_line_7:2:2: error: use of undeclared identifier 'it'
 it = liste.begin();   // Zur 3
 ^
input_line_7:2:7: error: use of undeclared identifier 'liste'
 it = liste.begin();   // Zur 3
      ^
input_line_7:3:1: error: use of undeclared identifier 'it'
it++;                 // Zur 2
^
input_line_7:4:1: error: use of undeclared identifier 'it2'
it2 = liste.erase(it);      // Die 2 wird gelöscht.
^
input_line_7:4:7: error: use of undeclared identifier 'liste'
it2 = liste.erase(it);      // Die 2 wird gelöscht.
      ^
input_line_7:4:19: error: use of undeclared identifier 'it'
it2 = liste.erase(it);      // Die 2 wird gelöscht.
                  ^
input_line_7:6:15: error: use of undeclared identifier 'liste'
for ( int i : liste ) {
              ^
input_line_7:7:5: error: use of undeclared identifier 'cout'
    cout << i << " ";
    ^


Interpreter Error: 

Wie angekündigt wurde die 2 gelöscht. Und wo steht der Iterator?

In [25]:
cout << *it;

32

In der Regel scheint der Iterator auf einen Wert zu verweisen, der gar nicht in der Liste war (durch Zufall kann es ein Wert aus der Liste sein). Tatsächlich wird der übergebene Iterator durch das Löschen
**ungültig**.

Aber wie sieht es mit dem zurückgegebenen Iterator `it2` aus?


In [26]:
cout << *it2;

4

Er zweigt auf das Element **hinter** dem gelöschten. D.h. bei `erase()` müssen wir darauf achten, dass der Iterator entsprechend gesetzt wird.

Wir verdeutlichen das Problem an einem Beispiel, bei dem wir aus den Zahlen von 1 bis 20 alle nicht durch drei teilbaren entfernen.

In [ ]:
#include <iostream>
#include <list>
using namespace std;

list<int> liste = list<int>();

// Befüllen der Liste
for ( int i = 1; i <= 20; i++ ) {
    liste.push_back(i);
}


auto it = liste.begin();

while ( it != liste.end() ) {
    // Lösche einen geraden  Wert
    if ( *it % 3 != 0 ) {
        liste.erase(it);
    }
    // Gehe zum nächsten
    it++;
}

for ( int i : liste ) {
    cout << i << " " ;
}

Führt man dieses Programm aus, stürzt es ab. Das Problem besteht darin, dass nach dem Löschen in Zeile 18 der Iterator `it` **ungültig** ist. Dadurch stürzt das Programm beim Erhöhen in Zeile 21 ab!

Um das zu beheben, müssen wir den von `erase()` zurückgegebenen Iterator nutzen.

In [3]:
#include <iostream>
#include <list>
using namespace std;

list<int> liste = list<int>();

// Befüllen der Liste
for ( int i = 1; i <= 20; i++ ) {
    liste.push_back(i);
}


auto it = liste.begin();

while ( it != liste.end() ) {
    // Lösche einen geraden  Wert
    if ( *it % 3 != 0 ) {
        // Nehme den zurückgergebenen Iterator
        it = liste.erase(it);
    }
    // Gehe zum nächsten
    it++;
}

for ( int i : liste ) {
    cout << i << " " ;
}

2 3 5 6 8 9 11 12 14 15 17 18 20 

Hmm. Das Programm ist nicht abgestürzt. Aber da ist etwas anderes schief gegangen. Wie es scheint wurden nicht alle nicht durch drei teilbaren Zahlen gelöscht. Versuchen wir nachzuvollziehen, was schief gegangen ist.

Wir haben mit der folgenden Liste angefangen: `1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20`

Der Iterator wurde anfänglich auf die `1` gesetzt. Da sie nicht durch drei teilbar ist, wird sie gelöscht. Das geschieht in Zeile 19. Gleichzeitig wird `it` auf den Rückgabewert von `erase()`gesetzt. Dies ist das Element **hinter** dem gelöschten, also `2`. Anschließend wird der Iterator in Zeile 22 nocheinmal erhöht und er steht anschließend auf `3`.

Die `2` wurde also übersprungen. Dadurch wird sie nicht geprüft und nicht gelöscht. Das gleiche geschieht mit der 5, der 8, der 11 und mit allen verbliebenen nicht durch drei teilbaren Zahlen.

In der Konsequenz müssen wird den Iterator nach dem Löschen nicht noch zusätzlich erhöhen, da dies durch `erase()` bereits geschehen ist. Also führen wir `it++` (Zeile 22) in der Alternative, dem `else`-Zweig aus.
    


In [4]:
#include <iostream>
#include <list>
using namespace std;

list<int> liste = list<int>();

// Befüllen der Liste
for ( int i = 1; i <= 20; i++ ) {
    liste.push_back(i);
}


auto it = liste.begin();

while ( it != liste.end() ) {
    // Lösche einen geraden  Wert
    if ( *it % 3 != 0 ) {
        // Nehme den zurückgergebenen Iterator
        it = liste.erase(it);
    } else {  // NEU!!!
        // Gehe zum nächsten
        it++;
    }
}

for ( int i : liste ) {
    cout << i << " " ;
}

3 6 9 12 15 18 

Jetzt klappt es!

Fassen wir zusammen, was wir gesehen haben: Wir ein Element aus einer Liste gelöscht, wird der Iterator **ungültig**. Allerdins gibt `erase` einen ITerator auf das Element **hinter** dem gelöschten zurück, so dass
man mit `it = liste.erase(it)`bereits beim nächsten Element ist. D.h. will man weiter laufen, muss man nicht noch zusätzlich den Iterator weiter setzen.

<div class="followup">
    <h3>Wo es weiter geht</h3>
        <div>
            Als Beispiel für die Arbeit mit Listen schauen wir uns in der nächsten Lesson das <a class="followup" href="/user-redirect/algoviz/lessons/06_ADT/04_SortiertesEinfuegen.ipynb">sortierte Einfügen an.</a>
    </div>
</div>    